In [3]:
import psycopg2
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
print(12)

12


In [4]:
conn = psycopg2.connect(
           dbname="qdap_test",
           user="amt",
           password="your_password",
           host="192.168.2.23",
           port="5432"
       )

# Create a cursor object using the cursor() method
cursor = conn.cursor()

def fetch_options(cursor, symbol, expiry):
    query = '''
        SELECT * 
        FROM ohlcv_options_per_minute oopm
        WHERE symbol = %s
        AND expiry_type = 'I'
        AND expiry = %s
        ORDER BY date_timestamp;
        '''
    cursor.execute(query,(symbol,expiry))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df
# Create a cursor object
cursor = conn.cursor()

def fetch_futures(cursor, symbol, x):
    query = '''
        SELECT *
        FROM ohlcv_future_per_minute ofpm 
        WHERE ofpm.symbol = %s
        AND ofpm.expiry_type = 'I'
        AND DATE(ofpm.expiry) = %s
        ORDER BY date_timestamp ASC
    '''
    # Execute the query with parameters as a tuple
    cursor.execute(query, (symbol, x))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

# Define the fetch_expiries function
def fetch_expiries(cursor, symbol):
    query = f'''
        SELECT DISTINCT ofpem.expiry 
        FROM ohlcv_future_per_minute ofpem 
        WHERE ofpem.symbol = '{symbol}'
        AND ofpem.expiry_type = 'I'
        GROUP BY ofpem.expiry 
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

In [5]:
symbol = 'BANKNIFTY'  # Replace with your symbol
expiry_future = fetch_expiries(cursor, symbol)

# Convert the 'expiry' column to datetime and then to date
expiry_future['expiry'] = pd.to_datetime(expiry_future['expiry']).dt.date

# Print the expiry dates with their indices
for i, row in expiry_future.iterrows():
    print(i, row.iloc[0], sep=' = ', end=' | ')
    if (i + 1) % 5 == 0:
        print()

0 = 2022-05-26 | 1 = 2022-06-30 | 2 = 2022-07-28 | 3 = 2022-08-25 | 4 = 2022-09-29 | 
5 = 2022-10-27 | 6 = 2022-11-24 | 7 = 2022-12-29 | 8 = 2023-01-25 | 9 = 2023-02-23 | 
10 = 2023-03-29 | 11 = 2023-04-27 | 12 = 2023-05-25 | 13 = 2023-06-28 | 14 = 2023-06-29 | 
15 = 2023-07-27 | 16 = 2023-08-31 | 17 = 2023-09-28 | 18 = 2023-10-26 | 19 = 2023-11-30 | 
20 = 2023-12-28 | 21 = 2024-01-25 | 22 = 2024-02-29 | 23 = 2024-03-27 | 24 = 2024-04-24 | 
25 = 2024-05-29 | 26 = 2024-06-26 | 27 = 2024-07-31 | 

In [35]:
index = 12

expiry_date = expiry_future.iloc[index]['expiry']

nft = fetch_futures(cursor, symbol, expiry_date)

nft.drop(columns=['id'])


,symbol,date_timestamp,open,low,high,close,volume,expiry,expiry_type
0,BANKNIFTY,2023-04-28 09:15:00,4305995,4304000,4313995,4312890,62525,2023-05-25 14:30:00,I
1,BANKNIFTY,2023-04-28 09:16:00,4313000,4310355,4313270,4310500,23100,2023-05-25 14:30:00,I
2,BANKNIFTY,2023-04-28 09:17:00,4310855,4310360,4312300,4310650,16600,2023-05-25 14:30:00,I
3,BANKNIFTY,2023-04-28 09:18:00,4310580,4309845,4311095,4309865,18875,2023-05-25 14:30:00,I
4,BANKNIFTY,2023-04-28 09:19:00,4310000,4304300,4310200,4304970,26400,2023-05-25 14:30:00,I
...,...,...,...,...,...,...,...,...,...
13870,BANKNIFTY,2023-05-25 15:27:00,4368500,4368000,4368600,4368200,6875,2023-05-25 14:30:00,I
13871,BANKNIFTY,2023-05-25 15:28:00,4368200,4368000,4368700,4368500,6950,2023-05-25 14:30:00,I
13872,BANKNIFTY,2023-05-25 15:28:00,4368200,4368000,4368700,4368500,6950,2023-05-25 14:30:00,I
13873,BANKNIFTY,2023-05-25 15:29:00,4368630,4368000,4368785,4368000,9700,2023-05-25 14:30:00,I


In [7]:

short_window = 9
long_window = 26
nft['Short_EMA'] = nft['close'].ewm(span=short_window, adjust=False).mean()
nft['Long_EMA'] = nft['close'].ewm(span=long_window, adjust=False).mean()

nft['Signal'] = 0

for i in range(1, len(nft)):
    if nft['Short_EMA'].iloc[i] > nft['Long_EMA'].iloc[i] and nft['Short_EMA'].iloc[i-1] <= nft['Long_EMA'].iloc[i-1]:
        nft.at[nft.index[i], 'Signal'] = 1  # Buy signal
    elif nft['Short_EMA'].iloc[i] < nft['Long_EMA'].iloc[i] and nft['Short_EMA'].iloc[i-1] >= nft['Long_EMA'].iloc[i-1]:
        nft.at[nft.index[i], 'Signal'] = -1  # Sell signal

nft.head()

,symbol,date_timestamp,open,low,high,close,volume,expiry,expiry_type,id,Short_EMA,Long_EMA,Signal
0,BANKNIFTY,2023-04-28 09:15:00,4305995,4304000,4313995,4312890,62525,2023-05-25 14:30:00,I,46716386,4312890.000,4.312890e+06,0
1,BANKNIFTY,2023-04-28 09:16:00,4313000,4310355,4313270,4310500,23100,2023-05-25 14:30:00,I,46716387,4312412.000,4.312713e+06,-1
2,BANKNIFTY,2023-04-28 09:17:00,4310855,4310360,4312300,4310650,16600,2023-05-25 14:30:00,I,46716388,4312059.600,4.312560e+06,0
3,BANKNIFTY,2023-04-28 09:18:00,4310580,4309845,4311095,4309865,18875,2023-05-25 14:30:00,I,46716389,4311620.680,4.312361e+06,0
4,BANKNIFTY,2023-04-28 09:19:00,4310000,4304300,4310200,4304970,26400,2023-05-25 14:30:00,I,46716390,4310290.544,4.311813e+06,0


In [8]:

fig = make_subplots(rows=1, cols=1)

# Candlestick chart
candlestick = go.Candlestick(x=nft.index,
                             open=nft['open'],
                             high=nft['high'],
                             low=nft['low'],
                             close=nft['close'],
                             name='Candlesticks')
fig.add_trace(candlestick)

# Short EMA
short_ema = go.Scatter(x=nft.index, y=nft['Short_EMA'], mode='lines', name='Short EMA')
fig.add_trace(short_ema)

# Long EMA
long_ema = go.Scatter(x=nft.index, y=nft['Long_EMA'], mode='lines', name='Long EMA')
fig.add_trace(long_ema)

# Buy signals
buy_signals = go.Scatter(x=nft[nft['Signal'] == 1].index, 
                         y=nft['Short_EMA'][nft['Signal'] == 1], 
                         mode='markers', 
                         marker=dict(symbol='triangle-up', color='blue', size=8), 
                         name='Buy Signal')
fig.add_trace(buy_signals)

# Sell signals
sell_signals = go.Scatter(x=nft[nft['Signal'] == -1].index, 
                          y=nft['Short_EMA'][nft['Signal'] == -1], 
                          mode='markers', 
                          marker=dict(symbol='triangle-down', color='black', size=8), 
                          name='Sell Signal')
fig.add_trace(sell_signals)

fig.update_layout(
    title=f'{symbol} EMA Crossover Strategy',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    width=2000,
    height=1000
)

pio.write_html(fig,file='p.html',auto_open=True)


In [9]:

position_size = 1  
current_position = 0  # Track current position (1 for long, -1 for short, 0 for neutral)
entry_price = 0  # Track entry price
total_pnl = 0  # Total P&L
# Iterate through each row in the DataFrame
for index, row in nft.iterrows():
    if row['Signal'] == 1 and current_position == 0:  # Buy signal and no current position
        current_position = 1
        # print("entry")
        entry_price = row['close']
    elif row['Signal'] == -1 and current_position == 1:  # Sell signal and long position
        pnl = (row['close'] - entry_price) * position_size
        total_pnl += pnl
        # ###print("close")
        current_position = 0
        entry_price = 0
    elif row['Signal'] == -1 and current_position == 0:  # Sell signal but no current long position
        continue  # Can't sell if not long

# Print total P&L
print(f'Total P&L: {total_pnl}')

Total P&L: 51180


In [10]:

# Assuming fetch_options is defined to fetch data based on 'symbol' and 'expiry'
cursor1 = conn.cursor()
options = fetch_options(cursor1, symbol=symbol, expiry=str(expiry_date))

In [31]:



k = options.copy()
k = k.drop(columns=['id'])

options.describe()

,date_timestamp,open,high,low,close,volume,strike,id
count,576978,5.769780e+05,5.769780e+05,5.769780e+05,5.769780e+05,5.769780e+05,5.769780e+05,5.769780e+05
mean,2023-05-15 18:55:41.234466816,3.341388e+04,3.357210e+04,3.325023e+04,3.341072e+04,2.657371e+04,4.290397e+06,8.289261e+06
min,2023-05-02 09:15:00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,2.500000e+01,3.350000e+06,6.000000e+00
25%,2023-05-09 15:16:00,9.500000e+02,9.600000e+02,9.400000e+02,9.500000e+02,1.500000e+02,4.160000e+06,4.194073e+06
50%,2023-05-17 09:43:00,7.510000e+03,7.600000e+03,7.415000e+03,7.505000e+03,9.250000e+02,4.320000e+06,8.848116e+06
75%,2023-05-22 14:05:00,4.148500e+04,4.186500e+04,4.104000e+04,4.147875e+04,5.925000e+03,4.440000e+06,1.193805e+07
max,2023-05-25 15:29:00,1.054990e+06,1.054990e+06,1.054990e+06,1.054990e+06,1.587715e+07,5.000000e+06,1.486146e+07
std,NaN,6.775641e+04,6.781407e+04,6.769299e+04,6.775019e+04,1.616819e+05,2.491164e+05,4.437196e+06


In [36]:
k.drop_duplicates()
k.describe()


,date_timestamp,open,high,low,close,volume,strike
count,576978,5.769780e+05,5.769780e+05,5.769780e+05,5.769780e+05,5.769780e+05,5.769780e+05
mean,2023-05-15 18:55:41.234466816,3.341388e+04,3.357210e+04,3.325023e+04,3.341072e+04,2.657371e+04,4.290397e+06
min,2023-05-02 09:15:00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,2.500000e+01,3.350000e+06
25%,2023-05-09 15:16:00,9.500000e+02,9.600000e+02,9.400000e+02,9.500000e+02,1.500000e+02,4.160000e+06
50%,2023-05-17 09:43:00,7.510000e+03,7.600000e+03,7.415000e+03,7.505000e+03,9.250000e+02,4.320000e+06
75%,2023-05-22 14:05:00,4.148500e+04,4.186500e+04,4.104000e+04,4.147875e+04,5.925000e+03,4.440000e+06
max,2023-05-25 15:29:00,1.054990e+06,1.054990e+06,1.054990e+06,1.054990e+06,1.587715e+07,5.000000e+06
std,NaN,6.775641e+04,6.781407e+04,6.769299e+04,6.775019e+04,1.616819e+05,2.491164e+05


In [33]:
copt = options[options['opt_type']=='CE'].copy()
popt = options[options['opt_type']=='PE'].copy()

In [13]:
# Initialize capital
initial_capital = 1000000  # Example initial capital
remaining_capital = initial_capital
profits = 0

put = {'pos': 0, 'dts': None, 'strike': None, 'buy_price': None}
call = {'pos': 0, 'dts': None, 'strike': None, 'buy_price': None}
trades = []
trade_details = []  # List to store buy and sell prices

print(type(nft.iloc[1]['date_timestamp']))

for i in range(len(nft) - 1):
    row = nft.iloc[i]
    signal = row['Signal']
    close = nft.iloc[i + 1]['close']
    dt = nft.iloc[i + 1]['date_timestamp']

    if signal == 1:
        print('s1', end='-> ')
        atm_call = copt[(copt['date_timestamp'] == dt) & (abs(copt['strike'] - close) == abs(copt['strike'] - close).min())]

        if not atm_call.empty:
            call_price = atm_call.iloc[0]['close']
            if remaining_capital >= call_price:
                print('long on call', end=' ')
                trades.append({
                    'position_type': 'long',
                    'opt_type': 'CE',
                    'price': call_price,
                    'day': dt.date()
                })
                call['pos'] = 1
                call['strike'] = atm_call.iloc[0]['strike']
                call['buy_price'] = call_price
                remaining_capital -= call_price  # Deduct the capital

            if put['pos'] == 1:
                put_short = popt[(popt['date_timestamp'] == dt) & (popt['strike'] == put['strike'])]
                if not put_short.empty:
                    print(' => short on put', end=' ')
                    put_sell_price = put_short.iloc[0]['close']
                    trades.append({
                        'position_type': 'short',
                        'opt_type': 'PE',
                        'price': put_sell_price,
                        'day': dt.date()
                    })
                    trade_details.append({
                        'type': 'PE',
                        'buy_price': put['buy_price'],
                        'sell_price': put_sell_price,
                        'pnl': put_sell_price - put['buy_price']
                    })
                    put['pos'] = 0
                    remaining_capital += put_sell_price  # Refill the capital
                    if remaining_capital > initial_capital:
                        profits += (remaining_capital - initial_capital)
                        remaining_capital = initial_capital
                else:
                    print('no closing on put opt ', end=' ')
        print('', end='\n')

    elif signal == -1:
        print('s2', end='-> ')
        atm_put = popt[(popt['date_timestamp'] == dt) & (abs(popt['strike'] - close) == abs(popt['strike'] - close).min())]

        if not atm_put.empty:
            put_price = atm_put.iloc[0]['close']
            if remaining_capital >= put_price:
                print('long on put ', end=' ')
                trades.append({
                    'position_type': 'long',
                    'opt_type': 'PE',
                    'price': put_price,
                    'day': dt.date()
                })
                put['pos'] = 1
                put['strike'] = atm_put.iloc[0]['strike']
                put['buy_price'] = put_price
                remaining_capital -= put_price  # Deduct the capital

            if call['pos'] == 1:
                call_short = copt[(copt['date_timestamp'] == dt) & (copt['strike'] == call['strike'])]
                if not call_short.empty:
                    print('short on call opt', end=' ')
                    call_sell_price = call_short.iloc[0]['close']
                    trades.append({
                        'position_type': 'short',
                        'opt_type': 'CE',
                        'price': call_sell_price,
                        'day': dt.date()
                    })
                    trade_details.append({
                        'type': 'CE',
                        'buy_price': call['buy_price'],
                        'sell_price': call_sell_price,
                        'pnl': call_sell_price - call['buy_price']
                    })
                    call['pos'] = 0
                    remaining_capital += call_sell_price  # Refill the capital
                    if remaining_capital > initial_capital:
                        profits += (remaining_capital - initial_capital)
                        remaining_capital = initial_capital
                else:
                    print('no closing on call opt ', end=' ')
        print('', end='\n')

if call['pos'] == 1:
    call_short = copt[(copt['date_timestamp'] == nft.iloc[-1]['date_timestamp']) & (copt['strike'] == call['strike'])]
    if not call_short.empty:
        print('short on call opt', end=' ')
        call_sell_price = call_short.iloc[0]['close']
        trades.append({
            'position_type': 'short',
            'opt_type': 'CE',
            'price': call_sell_price,
            'day': dt.date()
        })
        trade_details.append({
            'type': 'CE',
            'buy_price': call['buy_price'],
            'sell_price': call_sell_price,
            'pnl': call_sell_price - call['buy_price']
        })
        call['pos'] = 0
        remaining_capital += call_sell_price  # Refill the capital
        if remaining_capital > initial_capital:
            profits += (remaining_capital - initial_capital)
            remaining_capital = initial_capital
    else:
        print('no closing on call opt ', end=' -- ')

if put['pos'] == 1:
    put_short = popt[(popt['date_timestamp'] == dt) & (popt['strike'] == put['strike'])]
    if not put_short.empty:
        print(' => short on put', end=' ')
        put_sell_price = put_short.iloc[0]['close']
        trades.append({
            'position_type': 'short',
            'opt_type': 'PE',
            'price': put_sell_price,
            'day': dt.date()
        })
        trade_details.append({
            'type': 'PE',
            'buy_price': put['buy_price'],
            'sell_price': put_sell_price,
            'pnl': put_sell_price - put['buy_price']
        })
        put['pos'] = 0
        remaining_capital += put_sell_price  # Refill the capital
        if remaining_capital > initial_capital:
            profits += (remaining_capital - initial_capital)
            remaining_capital = initial_capital
else:
    print('no closing on put opt ', end=' --\n')

# Print final capital and profits
print(f"Final remaining capital: {remaining_capital}")
print(f"Total profits: {profits}")


<class 'pandas._libs.tslibs.timestamps.Timestamp'>
s2-> 
s1-> 
s2-> 
s1-> 
s2-> 
s1-> 
s2-> 
s1-> 
s2-> 
s1-> 
s2-> long on put  
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call opt 
s1-> long on call  => short on put 
s2-> long on put  short on call

In [14]:

# Print trade details
print("\nTrade Details:")
for detail in trade_details:
    print(detail['pnl'])



Trade Details:
-1495
-1575
-445
-2355
-955
85
-870
2050
-235
-60
665
-810
-725
-230
190
-725
-395
-275
420
360
-1000
-670
-340
-450
1055
-145
-325
-1120
11160
2055
-195
3890
1745
0
-415
1685
-800
-505
500
-740
1500
-1225
-340
-315
-130
-430
-1200
-575
-750
-975
1680
-705
90
390
460
0
660
-2590
-2460
-1680
-875
-670
-755
330
1820
-620
405
-445
-5
-825
-780
710
0
-1360
-610
-485
320
-290
-760
-215
975
45
0
-135
1855
0
-640
240
165
-1760
10215
8895
-1320
-390
6430
710
-865
14985
300
7660
2075
-3145
-960
-1955
-225
-2095
-3470
-1555
-1810
8645
-820
-1085
0
940
0
-2760
-2565
-230
-3395
5870
-3565
-9390
20700
-2045
-990
-325
0
-1140
50
-555
-1200
250
0
-1240
1320
-1040
-1260
-1680
-655
-655
-1200
5
-1200
-1565
-5115
-3000
-670
-920
-510
-1040
1380
-940
-1525
715
3130
-305
540
-725
-845
-1800
-120
-820
-530
-5
50
450
480
0
1970
-500
-1470
-1690
-85
5350
-795
6605
-1265
-810
-1030
-2015
-140
720
400
0
-290
2550
290
17815
0
-650
3315
-2120
7375
-240
-1305
-1825
-540
15
-630
-2510
-2370
3470
63

In [15]:

# Extract PnL values
pnl_values = [detail['pnl'] for detail in trade_details]

# Create a Plotly plot
trace = go.Scatter(
    x=list(range(len(pnl_values))),
    y=pnl_values,
    mode='lines+markers',
    name='PnL'
)

layout = go.Layout(
    title='Trade PnL Over Time',
    xaxis=dict(title='Trade Number'),
    yaxis=dict(title='PnL'),
    hovermode='closest'
)

fig = go.Figure(data=[trace], layout=layout)
pyo.plot(fig, auto_open=True)

'temp-plot.html'

In [16]:
pnl_values = [detail['pnl'] for detail in trade_details]
# Assume risk-free rate (R_f) is 0 for simplicity, or define it as needed
risk_free_rate = 1.0

# Calculate average return (R_p)
average_return = np.mean(pnl_values)

# Calculate standard deviation of returns (σ_p)
std_dev_return = np.std(pnl_values)

# Calculate Sharpe Ratio
sharpe_ratio = (average_return - risk_free_rate) / std_dev_return

print(f"Average Return (R_p): {average_return}")
print(f"Standard Deviation of Returns (σ_p): {std_dev_return}")
print(f"Sharpe Ratio: {sharpe_ratio}")


Average Return (R_p): 81.6696113074205
Standard Deviation of Returns (σ_p): 2636.4312756847435
Sharpe Ratio: 0.030598033049986744


In [17]:
daily_expenses = {}
daily_earnings = {}

for trade in trades:
    day = trade['day']
    price = trade['price']
    position_type = trade['position_type']

    if position_type == 'long':
        # Update daily expenses
        if day not in daily_expenses:
            daily_expenses[day] = 0
        daily_expenses[day] += price
    else:
        # Update daily earnings
        if day not in daily_earnings:
            daily_earnings[day] = 0
        daily_earnings[day] += price


In [18]:
net_profit = {}

# print("Daily Expenses:")
for day, expense in daily_expenses.items():
    if day not in net_profit:
        net_profit[day]=0
    net_profit[day]-=expense
    # print(f"{day}: {expense}")

# print("Daily Earnings:")
for day, earning in daily_earnings.items():
    if day not in net_profit:
        net_profit[day]=0
    net_profit[day]-=earning
    # print(f"{day}: {earning}")

for day , expences in net_profit.items():
    print(f"on {day} : {expences}")


on 2023-05-02 : -3211425
on 2023-05-03 : -3301000
on 2023-05-04 : -3649780
on 2023-05-05 : -3191205
on 2023-05-08 : -3373320
on 2023-05-09 : -3079185
on 2023-05-10 : -2073405
on 2023-05-11 : -3473045
on 2023-05-12 : -3258325
on 2023-05-15 : -2707420
on 2023-05-16 : -2653005
on 2023-05-17 : -2179095
on 2023-05-18 : -1791190
on 2023-05-19 : -1748360
on 2023-05-22 : -1987615
on 2023-05-23 : -1239340
on 2023-05-24 : -795445
on 2023-05-25 : -466505


In [19]:

def calculate_drawdown(values):
    peak = -1000000000000
    dr = 0
    i, n = 0, len(values)
    
    while i < n:
        if values[i] > peak:
            peak = values[i]
            while i < n and values[i] <= peak:
                dr = max(dr, abs(peak - values[i]))
                i += 1
        else:
            i += 1
            
    return dr

# Convert dict_values to a list
arr = list(net_profit.values())

print("Net Profit Values:", arr)
print("Maximum Drawdown:", calculate_drawdown(arr))

Net Profit Values: [np.int64(-3211425), np.int64(-3301000), np.int64(-3649780), np.int64(-3191205), np.int64(-3373320), np.int64(-3079185), np.int64(-2073405), np.int64(-3473045), np.int64(-3258325), np.int64(-2707420), np.int64(-2653005), np.int64(-2179095), np.int64(-1791190), np.int64(-1748360), np.int64(-1987615), np.int64(-1239340), np.int64(-795445), np.int64(-466505)]
Maximum Drawdown: 1399640


In [20]:

# Plotting with Plotly
trace = go.Scatter(
    x=list(net_profit.keys()),  # Dates as x-axis
    y=arr,                      # Net profit values as y-axis
    mode='lines+markers',       # Line plot with markers
    name='Net Profit'
)

layout = go.Layout(
    title='Net Profit Over Time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Net Profit'),
    hovermode='closest'
)

fig = go.Figure(data=[trace], layout=layout)
pyo.plot(fig, auto_open=True)

'temp-plot.html'

In [21]:
pnl = 0
day_trades = []
for it in trades:
    
    print(f"{it['position_type']} on {it['opt_type']} with strike price = {it['price']} on date {it['day']}")
    if it['position_type']=='long':
        pnl -= it['price']
    else:
        pnl += it['price']
print(pnl)



long on PE with strike price = 56245 on date 2023-05-02
long on CE with strike price = 58075 on date 2023-05-02
short on PE with strike price = 54750 on date 2023-05-02
long on PE with strike price = 56735 on date 2023-05-02
short on CE with strike price = 56500 on date 2023-05-02
long on CE with strike price = 56335 on date 2023-05-02
short on PE with strike price = 56290 on date 2023-05-02
long on PE with strike price = 58100 on date 2023-05-02
short on CE with strike price = 53980 on date 2023-05-02
long on CE with strike price = 55600 on date 2023-05-02
short on PE with strike price = 57145 on date 2023-05-02
long on PE with strike price = 56570 on date 2023-05-02
short on CE with strike price = 55685 on date 2023-05-02
long on CE with strike price = 56665 on date 2023-05-02
short on PE with strike price = 55700 on date 2023-05-02
long on PE with strike price = 54465 on date 2023-05-02
short on CE with strike price = 58715 on date 2023-05-02
long on CE with strike price = 58800 on 

In [22]:
# Initialize PnL and a list to track PnL over time
pnl = 0
pnl_changes = []


# Iterate over trades to calculate PnL per trade
for it in trades:
    print(f"{it['position_type']} on {it['opt_type']} with strike price = {it['price']}")
    if it['position_type'] == 'long':
        pnl_change = -it['price']
    else:
        pnl_change = it['price']
    pnl_changes.append(pnl_change)  # Track PnL change per trade
    pnl += pnl_change

# Print final PnL
print(f"Final PnL: {pnl}")

# Plotting PnL changes per trade with Plotly
trace = go.Bar(
    x=list(range(1, len(pnl_changes) + 1)),
    y=pnl_changes,
    name='PnL Change Per Trade'
)

layout = go.Layout(
    title='PnL Change Per Trade',
    xaxis=dict(title='Trade Number'),
    yaxis=dict(title='PnL Change'),
    hovermode='closest'
)

fig = go.Figure(data=[trace], layout=layout)
pyo.plot(fig, auto_open=True)

long on PE with strike price = 56245
long on CE with strike price = 58075
short on PE with strike price = 54750
long on PE with strike price = 56735
short on CE with strike price = 56500
long on CE with strike price = 56335
short on PE with strike price = 56290
long on PE with strike price = 58100
short on CE with strike price = 53980
long on CE with strike price = 55600
short on PE with strike price = 57145
long on PE with strike price = 56570
short on CE with strike price = 55685
long on CE with strike price = 56665
short on PE with strike price = 55700
long on PE with strike price = 54465
short on CE with strike price = 58715
long on CE with strike price = 58800
short on PE with strike price = 54230
long on PE with strike price = 54150
short on CE with strike price = 58740
long on CE with strike price = 58405
short on PE with strike price = 54815
long on PE with strike price = 55415
short on CE with strike price = 57595
long on CE with strike price = 58225
short on PE with strike pr

'temp-plot.html'